# MANDATE - 3 (MT2021069)

## In this mandate, I have applied Root Mapping on the Context, its Sentences and Questions and tried to match the root of question to the roots of sentences in the context and based on that tried to find the most probable answer in the context for the given question.

## In the last Mandate, I had applied Unsupervised techniques on the Embedded QA Dataset using Cosine Similarity and Euclidean Distances. In this Mandate, I have tried various Supervised Algorithms to increase the Accuracy.

## Importing Necessary Libraries and Packages :

In [14]:
import numpy as np, pandas as pd
import ast 
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import spacy
from nltk import Tree
en_nlp = spacy.load('en_core_web_sm')
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

### Reading the csv generated in last mandate:

#### In this dataframe we have the sentence embeddings and question embeddings along with the Cosine Similarity and Euclidean distances of the question with respect to all the sentences in the context. It also includes the predicted sentence number in the context which couble be the probable answer for the question, according to the unsupervised techniques applied earlier.

In [15]:
predicted = pd.read_csv(r'C:\Users\admin\Downloads\predictedNLP1.csv').reset_index(drop=True)

In [16]:
predicted

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,pred_idx_cos,pred_idx_euc
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"['Architecturally, the school has a Catholic c...",5,"[array([-0.01541213, -0.05413497, 0.01449186,...",[[ 0.00746889 0.02421027 0.06961634 ... 0.0...,"[0.7483252286911011, 0.6157153248786926, 0.616...","[8.731496, 7.767769, 8.243154, 7.9353786, 7.73...",5,5
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"['Architecturally, the school has a Catholic c...",2,"[array([-0.01541213, -0.05413497, 0.01449186,...",[[ 0.00746889 -0.03348338 0.04054591 ... -0.0...,"[0.7264537513256073, 0.5839367210865021, 0.654...","[7.0554705, 6.2977524, 7.563771, 4.9111557, 6....",3,3
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"['Architecturally, the school has a Catholic c...",3,"[array([-0.01541213, -0.05413497, 0.01449186,...",[[ 0.00746889 -0.04394472 0.1438594 ... 0.0...,"[0.6630303263664246, 0.5432414710521698, 0.575...","[6.4743433, 5.8934736, 6.712471, 4.1032743, 4....",3,3
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,"['Architecturally, the school has a Catholic c...",4,"[array([-0.01541213, -0.05413497, 0.01449186,...",[[ 0.00746889 -0.02113509 0.08985032 ... 0.0...,"[0.7390832602977753, 0.6787103712558746, 0.749...","[7.6907616, 7.736262, 9.104893, 5.9597845, 6.1...",3,3
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,"['Architecturally, the school has a Catholic c...",1,"[array([-0.01541213, -0.05413497, 0.01449186,...",[[ 0.00746889 -0.02515129 0.08926862 ... 0.0...,"[0.8073934763669968, 0.5599293112754822, 0.630...","[8.670769, 6.5675135, 7.8863955, 6.087176, 6.8...",1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
87593,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,229,Oregon,"['Kathmandu Metropolitan City (KMC), in order ...",1,"[array([-0.01553709, -0.07280491, 0.11549293,...",[[ 0.00746889 -0.03403048 0.06046114 ... 0.0...,"[0.45670151710510254, 0.4772855043411255, 0.52...","[5.9728227, 5.8767605, 7.1701756, 7.343958]",0,1
87594,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,414,Rangoon,"['Kathmandu Metropolitan City (KMC), in order ...",2,"[array([-0.01553709, -0.07280491, 0.11549293,...",[[ 0.00746889 -0.05823731 -0.00707996 ... -0.0...,"[0.7831550389528275, 0.7901124805212021, 0.798...","[9.570372, 9.148212, 10.2699175, 11.297274]",0,1
87595,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,476,Minsk,"['Kathmandu Metropolitan City (KMC), in order ...",2,"[array([-0.01553709, -0.07280491, 0.11549293,...",[[ 0.00746889 -0.05044248 0.06999452 ... 0.0...,"[0.4997228980064392, 0.5831607580184937, 0.566...","[6.429439, 7.027028, 7.642034, 7.855676]",0,0
87596,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,199,1975,"['Kathmandu Metropolitan City (KMC), in order ...",1,"[array([-0.01553709, -0.07280491, 0.11549293,...",[[ 0.00746889 -0.03403048 0.05538648 ... 0.0...,"[0.5239468514919281, 0.5166756808757782, 0.559...","[6.7326083, 6.274422, 7.5743055, 7.709353]",1,1


## Visualizing the root mapping :

### Function to form sentence root tree :

In [17]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

#### Taking 1st question for demo :

In [18]:
predicted.iloc[0,1]

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

#### Generated root map tree for 1st question :

In [19]:
[to_nltk_tree(sent.root).pretty_print()  for sent in en_nlp(predicted.iloc[0,1]).sents]

                  appear                             
  __________________|____________________________     
 |      |      |    |         |           |      in  
 |      |      |    |         |           |      |    
 |      |      |    To       Mary         in   France
 |      |      |    |      ___|_____      |      |    
did allegedly  ?   whom  the      Virgin 1858 Lourdes



[None]

#### Similarly, performing demo on context (1st paragraph) :

In [20]:
document = en_nlp(predicted.iloc[0,0])

In [21]:
predicted.iloc[0,0]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

#### Generated Root map trees for first 5 sentences of the paragraph :

In [22]:
[to_nltk_tree(sent.root) .pretty_print() for sent in document.sents][5]

                    has                              
        _____________|_________________               
       |         |   |  school     character         
       |         |   |    |      ______|________      
Architecturally  ,   .   the    a            Catholic

                          is                                  
  ________________________|_______________                     
 |    |            |                    statue                
 |    |            |                  ____|_________           
 |    |           dome               |    |         of        
 |    |    ________|______           |    |         |          
 |    |   |            Building      |    |        Mary       
 |    |   |     __________|______    |    |      ___|_____     
Atop  .  gold the        Main    's  a  golden the      Virgin

                                                is                                                     
      __________________________________________|__

#### Identified roots & subroots of the sentences in the first paragraph after performing stemming on it.

In [23]:
for sent in document.sents:
    roots = [st.stem(chunk.root.head.text.lower()) for chunk in sent.noun_chunks]
    print(roots)

['has', 'has']
['is', 'is', 'of']
['in', 'of', 'fac', 'is', 'of', 'with', 'with', 'legend']
['to', 'is', 'of']
['behind', 'is', 'grotto', 'of', 'pray']
['is', 'is', 'of', 'at', 'lourd', 'appear', 'to']
['at', 'of', 'in', 'connect', 'through', 'statu', 'is', 'of']


## Function for Matching the Roots :

### Matching the roots of question and sentences in corresponding context

In [24]:
def match_roots(x):
    question = x["question"].lower()
    sentences = en_nlp(x["context"].lower()).sents
    
    question_root = st.stem(str([sent.root for sent in en_nlp(question).sents][0]))
    
    li = []
    for i,sent in enumerate(sentences):
        roots = [st.stem(chunk.root.head.text.lower()) for chunk in sent.noun_chunks]

        if question_root in roots: 
            for k,item in enumerate(ast.literal_eval(x["sentences"])):
                if str(sent) in item.lower(): 
                    li.append(k)
    return li

In [25]:
predicted["root_match_idx"] = predicted.apply(match_roots, axis = 1)

"root_match_idx" is a list of sentences in the context whose root matched with the root of the question.

In [26]:
predicted["root_match_idx_first"]= predicted["root_match_idx"].apply(lambda x: x[0] if len(x)>0 else 0)

"root_match_idx_first" is the first sentence in the context whose root matched with the root of the question.

In [28]:
predicted

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,pred_idx_cos,pred_idx_euc,root_match_idx,root_match_idx_first
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"['Architecturally, the school has a Catholic c...",5,"[array([-0.01541213, -0.05413497, 0.01449186,...",[[ 0.00746889 0.02421027 0.06961634 ... 0.0...,"[0.7483252286911011, 0.6157153248786926, 0.616...","[8.731496, 7.767769, 8.243154, 7.9353786, 7.73...",5,5,[5],5
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"['Architecturally, the school has a Catholic c...",2,"[array([-0.01541213, -0.05413497, 0.01449186,...",[[ 0.00746889 -0.03348338 0.04054591 ... -0.0...,"[0.7264537513256073, 0.5839367210865021, 0.654...","[7.0554705, 6.2977524, 7.563771, 4.9111557, 6....",3,3,"[1, 2, 3, 4, 5, 6]",1
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"['Architecturally, the school has a Catholic c...",3,"[array([-0.01541213, -0.05413497, 0.01449186,...",[[ 0.00746889 -0.04394472 0.1438594 ... 0.0...,"[0.6630303263664246, 0.5432414710521698, 0.575...","[6.4743433, 5.8934736, 6.712471, 4.1032743, 4....",3,3,"[1, 2, 3, 4, 5, 6]",1
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,"['Architecturally, the school has a Catholic c...",4,"[array([-0.01541213, -0.05413497, 0.01449186,...",[[ 0.00746889 -0.02113509 0.08985032 ... 0.0...,"[0.7390832602977753, 0.6787103712558746, 0.749...","[7.6907616, 7.736262, 9.104893, 5.9597845, 6.1...",3,3,"[1, 2, 3, 4, 5, 6]",1
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,"['Architecturally, the school has a Catholic c...",1,"[array([-0.01541213, -0.05413497, 0.01449186,...",[[ 0.00746889 -0.02515129 0.08926862 ... 0.0...,"[0.8073934763669968, 0.5599293112754822, 0.630...","[8.670769, 6.5675135, 7.8863955, 6.087176, 6.8...",1,3,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87593,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,229,Oregon,"['Kathmandu Metropolitan City (KMC), in order ...",1,"[array([-0.01553709, -0.07280491, 0.11549293,...",[[ 0.00746889 -0.03403048 0.06046114 ... 0.0...,"[0.45670151710510254, 0.4772855043411255, 0.52...","[5.9728227, 5.8767605, 7.1701756, 7.343958]",0,1,"[0, 1, 2]",0
87594,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,414,Rangoon,"['Kathmandu Metropolitan City (KMC), in order ...",2,"[array([-0.01553709, -0.07280491, 0.11549293,...",[[ 0.00746889 -0.05823731 -0.00707996 ... -0.0...,"[0.7831550389528275, 0.7901124805212021, 0.798...","[9.570372, 9.148212, 10.2699175, 11.297274]",0,1,[],0
87595,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,476,Minsk,"['Kathmandu Metropolitan City (KMC), in order ...",2,"[array([-0.01553709, -0.07280491, 0.11549293,...",[[ 0.00746889 -0.05044248 0.06999452 ... 0.0...,"[0.4997228980064392, 0.5831607580184937, 0.566...","[6.429439, 7.027028, 7.642034, 7.855676]",0,0,[],0
87596,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,199,1975,"['Kathmandu Metropolitan City (KMC), in order ...",1,"[array([-0.01553709, -0.07280491, 0.11549293,...",[[ 0.00746889 -0.03403048 0.05538648 ... 0.0...,"[0.5239468514919281, 0.5166756808757782, 0.559...","[6.7326083, 6.274422, 7.5743055, 7.709353]",1,1,[],0


In [30]:
predicted.to_csv(r'C:\Users\Admin\Downloads\predictedNLP2.csv', index=False)

## This root match technique was just implemented for experimenting and trying to find any new results. As the above table shows, in the root_match_idx column the list of sentences that have the same roots as the question, includes the answer predicted using cosine and Euclidean distances.

## But for many questions in the table, it is unable to find the same roots in the sentences of the context. So, I personally didn't found this approach more effective than cosine similarity technique.

# Applying Supervised Algorithms :

## Importing Necessary Libraries :

In [31]:
import numpy as np, pandas as pd
import ast 
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import spacy
from nltk import Tree
en_nlp = spacy.load('en_core_web_sm')
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

### Reading previously generated .csv :

In [32]:
data = pd.read_csv(r'C:\Users\admin\Downloads\predictedNLP2.csv').reset_index(drop=True)

In [33]:
data.shape

(87598, 14)

In [34]:
data.head(3)

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,pred_idx_cos,pred_idx_euc,root_match_idx,root_match_idx_first
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"['Architecturally, the school has a Catholic c...",5,"[array([-0.01541213, -0.05413497, 0.01449186,...",[[ 0.00746889 0.02421027 0.06961634 ... 0.0...,"[0.7483252286911011, 0.6157153248786926, 0.616...","[8.731496, 7.767769, 8.243154, 7.9353786, 7.73...",5,5,[5],5
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"['Architecturally, the school has a Catholic c...",2,"[array([-0.01541213, -0.05413497, 0.01449186,...",[[ 0.00746889 -0.03348338 0.04054591 ... -0.0...,"[0.7264537513256073, 0.5839367210865021, 0.654...","[7.0554705, 6.2977524, 7.563771, 4.9111557, 6....",3,3,"[1, 2, 3, 4, 5, 6]",1
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"['Architecturally, the school has a Catholic c...",3,"[array([-0.01541213, -0.05413497, 0.01449186,...",[[ 0.00746889 -0.04394472 0.1438594 ... 0.0...,"[0.6630303263664246, 0.5432414710521698, 0.575...","[6.4743433, 5.8934736, 6.712471, 4.1032743, 4....",3,3,"[1, 2, 3, 4, 5, 6]",1


### Restricting the paragraph (context) length to 10 sentences only, to maintain uniformity :

In [35]:
data = data[data["sentences"].apply(lambda x: len(ast.literal_eval(x)))<11].reset_index(drop=True)

## Creating a dataframe consisting of only Euclidean Distances, Cosine Similarities and target (Target line in paragraph which should be the answer) :

In [36]:
def create_features(data):
    train = pd.DataFrame()
     
    for k in range(len(data["euclidean_dis"])):
        dis = ast.literal_eval(data["euclidean_dis"][k])
        for i in range(len(dis)):
            train.loc[k, "column_euc_"+"%s"%i] = dis[i]
    
    print("Finished")
    
    for k in range(len(data["cosine_sim"])):
        dis = ast.literal_eval(data["cosine_sim"][k].replace("nan","1"))
        for i in range(len(dis)):
            train.loc[k, "column_cos_"+"%s"%i] = dis[i]
            
    train["target"] = data["target"]
    return train

In [37]:
train = create_features(data)

Finished


In [38]:
train.head(3)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9,...,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,8.731496,7.767769,8.243154,7.935379,7.735842,4.640070,8.041734,NaN,NaN,NaN,...,0.615715,0.616862,0.676589,0.675969,0.317759,0.614743,NaN,NaN,NaN,5
1,7.055471,6.297752,7.563771,4.911156,6.279219,8.261658,7.600303,NaN,NaN,NaN,...,0.583937,0.654297,0.500688,0.663226,0.649799,0.676015,NaN,NaN,NaN,2
2,6.474343,5.893474,6.712471,4.103274,4.880681,7.145757,6.512111,NaN,NaN,NaN,...,0.543241,0.575311,0.415578,0.512337,0.553865,0.573560,NaN,NaN,NaN,3


In [39]:
train.shape

(85129, 21)

In [40]:
train.head(3).transpose()

,0,1,2
column_euc_0,8.731496,7.055471,6.474343
column_euc_1,7.767769,6.297752,5.893474
column_euc_2,8.243154,7.563771,6.712471
column_euc_3,7.935379,4.911156,4.103274
column_euc_4,7.735842,6.279219,4.880681
column_euc_5,4.640070,8.261658,7.145757
column_euc_6,8.041734,7.600303,6.512111
column_euc_7,NaN,NaN,NaN
column_euc_8,NaN,NaN,NaN
column_euc_9,NaN,NaN,NaN


## Fitting Multinomial Logistic Regression

### Standardize

In [41]:
train.apply(max, axis = 0)

column_euc_0    32.503560
column_euc_1    30.244087
column_euc_2    29.838234
column_euc_3    27.500748
column_euc_4    28.242474
column_euc_5    25.417771
column_euc_6    28.922787
column_euc_7          NaN
column_euc_8          NaN
column_euc_9          NaN
column_cos_0     1.524448
column_cos_1     1.512856
column_cos_2     1.481605
column_cos_3     1.424789
column_cos_4     1.468127
column_cos_5     1.380353
column_cos_6     1.447956
column_cos_7          NaN
column_cos_8          NaN
column_cos_9          NaN
target           9.000000
dtype: float64

## Replace NaN of Euclidean distance with 60 and of Cosine similarity with 1 :

In [42]:
subset1 = train.iloc[:,:10].fillna(60)
subset2 = train.iloc[:,10:].fillna(1)

In [43]:
subset1.head(3)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9
0,8.731496,7.767769,8.243154,7.935379,7.735842,4.640070,8.041734,60.0,60.0,60.0
1,7.055471,6.297752,7.563771,4.911156,6.279219,8.261658,7.600303,60.0,60.0,60.0
2,6.474343,5.893474,6.712471,4.103274,4.880681,7.145757,6.512111,60.0,60.0,60.0


In [44]:
 subset2.head(3)

,column_cos_0,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,0.748325,0.615715,0.616862,0.676589,0.675969,0.317759,0.614743,1.0,1.0,1.0,5
1,0.726454,0.583937,0.654297,0.500688,0.663226,0.649799,0.676015,1.0,1.0,1.0,2
2,0.663030,0.543241,0.575311,0.415578,0.512337,0.553865,0.573560,1.0,1.0,1.0,3


In [45]:
train2 = pd.concat([subset1, subset2], axis=1)
train2 = train2.reindex(subset1.index)

In [46]:
train2.head(3)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9,...,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,8.731496,7.767769,8.243154,7.935379,7.735842,4.640070,8.041734,60.0,60.0,60.0,...,0.615715,0.616862,0.676589,0.675969,0.317759,0.614743,1.0,1.0,1.0,5
1,7.055471,6.297752,7.563771,4.911156,6.279219,8.261658,7.600303,60.0,60.0,60.0,...,0.583937,0.654297,0.500688,0.663226,0.649799,0.676015,1.0,1.0,1.0,2
2,6.474343,5.893474,6.712471,4.103274,4.880681,7.145757,6.512111,60.0,60.0,60.0,...,0.543241,0.575311,0.415578,0.512337,0.553865,0.573560,1.0,1.0,1.0,3


In [47]:
train2.apply(max, axis = 0)

column_euc_0    32.503560
column_euc_1    60.000000
column_euc_2    60.000000
column_euc_3    60.000000
column_euc_4    60.000000
column_euc_5    60.000000
column_euc_6    60.000000
column_euc_7    60.000000
column_euc_8    60.000000
column_euc_9    60.000000
column_cos_0     1.524448
column_cos_1     1.512856
column_cos_2     1.481605
column_cos_3     1.424789
column_cos_4     1.468127
column_cos_5     1.380353
column_cos_6     1.447956
column_cos_7     1.391503
column_cos_8     1.351021
column_cos_9     1.183319
target           9.000000
dtype: float64

In [48]:
scaler = MinMaxScaler()
X = scaler.fit_transform(train2.iloc[:,:-1])

In [49]:
X

array([[0.23175121, 0.10455141, 0.11096877, ..., 0.71864748, 0.7401808 ,
        0.84508039],
       [0.1775866 , 0.07935003, 0.09929896, ..., 0.71864748, 0.7401808 ,
        0.84508039],
       [0.15880614, 0.07241923, 0.08467611, ..., 0.71864748, 0.7401808 ,
        0.84508039],
       ...,
       [0.15735495, 0.09185244, 0.10064329, ..., 0.71864748, 0.7401808 ,
        0.84508039],
       [0.16715256, 0.07895006, 0.09947991, ..., 0.71864748, 0.7401808 ,
        0.84508039],
       [0.25125221, 0.12124778, 0.16662005, ..., 0.71864748, 0.7401808 ,
        0.84508039]])

### Train-test split :

In [50]:
train_x, test_x, train_y, test_y = train_test_split(X,
train.iloc[:,-1], train_size=0.8, random_state = 5)

### Train_x consists of Euclidean distances and cosine similarities of each question-context pair :

In [51]:
train_x

array([[0.18229271, 0.09514955, 0.08312137, ..., 0.71864748, 0.7401808 ,
        0.84508039],
       [0.26908418, 0.13444728, 0.10019235, ..., 0.71864748, 0.7401808 ,
        0.84508039],
       [0.33275775, 0.10497898, 0.11312171, ..., 0.71864748, 0.7401808 ,
        0.84508039],
       ...,
       [0.19970291, 0.12266808, 0.14424258, ..., 0.71864748, 0.7401808 ,
        0.84508039],
       [0.10196813, 1.        , 1.        , ..., 0.71864748, 0.7401808 ,
        0.84508039],
       [0.28535632, 0.14244074, 0.13023796, ..., 0.71864748, 0.7401808 ,
        0.84508039]])

### Train_y consists of target line no. that should be the answer to the question in the corresponding context :

In [52]:
train_y

295      2
1551     0
1610     0
5579     0
16167    0
        ..
59465    0
5520     3
20463    1
18638    0
35683    0
Name: target, Length: 68103, dtype: int64

In [53]:
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')
mul_lr.fit(train_x, train_y)

print("Multinomial Logistic regression Train Accuracy : ", metrics.accuracy_score(train_y, mul_lr.predict(train_x)))
print("Multinomial Logistic regression Test Accuracy : ", metrics.accuracy_score(test_y, mul_lr.predict(test_x)))

Multinomial Logistic regression Train Accuracy :  0.5734402302394902
Multinomial Logistic regression Test Accuracy :  0.5757664748032421


## Random Forest

In [61]:
rf = RandomForestClassifier(min_samples_leaf=8, n_estimators=60)
rf.fit(train_x, train_y)

print("Random Forest Train Accuracy : ", metrics.accuracy_score(train_y, rf.predict(train_x)))
print("Random Forest Test Accuracy : ", metrics.accuracy_score(test_y, rf.predict(test_x)))

Random Forest Train Accuracy :  0.7607594379102243
Random Forest Test Accuracy :  0.6491248678491719


## XgBoost

In [58]:
model = xgb.XGBClassifier()
param_dist = {"max_depth": [3,5,10],
              "min_child_weight" : [1,5,10],
              "learning_rate": [0.07, 0.1,0.2],
               }

# run randomized search
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
                                   verbose=5, n_jobs=-1)
grid_search.fit(train_x, train_y)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[23:16:43] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight

In [59]:
grid_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.2, max_delta_step=0,
              max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [60]:
xg = xgb.XGBClassifier(max_depth=5)
xg.fit(train_x, train_y)

print("XG_Boost Train Accuracy : ", metrics.accuracy_score(train_y, xg.predict(train_x)))
print("XG_Boost Test Accuracy : ", metrics.accuracy_score(test_y, xg.predict(test_x)))

[23:20:29] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XG_Boost Train Accuracy :  0.7469274481300383
XG_Boost Test Accuracy :  0.678432984846705


# 

# So following is the summary of the above Experimentations :

## For the case of Unsupervised techniques, Cosine similarity gives the best results than Euclidean distance method and Roots Match Method.

## For the case of Supervised techniques, following are the results :
### Multinomial Logistic regression Train Accuracy : 57.34%
### Multinomial Logistic regression Test Accuracy :  57.57%
###
### Random Forest Train Accuracy : 76.07%
### Random Forest Test Accuracy : 64.91%
###
### XG_Boost Train Accuracy : 74.69%
### XG_Boost Test Accuracy : 67.84%

## The results that I got through these experimentations are satisfactory as the main aim was to learn generating a QA model using SQuAD dataset, so that I can replicate it on the 'Bhagwad Gita'. 

## The next Mandate is the most Interesting Mandate, as I have tried to replicate the knowlegde from these experimentations on the 'Bhagwad Gita' dataset to predict the most relevant shlokas for any general question trained on the model.